In [4]:

# Taken grade as a Y variable
# Taken normalized values of numerial data as X 

#f-score: Training set given as csv:0.81
#f-score: Testing set given as csv:0.93

from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import datetime
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from pandas.plotting import scatter_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from datetime import datetime, date, time, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
plt.style.use('ggplot')



C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [5]:
#Reading csv file -training and testing data to separate dataframes.
df_training=pd.read_csv('train_indessa.csv')
df_testing=pd.read_csv('test_indessa.csv')
df_training.columns
df_testing.columns

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'batch_enrolled', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'application_type', 'verification_status_joint', 'last_week_pay',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim'],
      dtype='object')

In [6]:
#Cleaning Training data
df_training.drop(['batch_enrolled'], axis = 1)
df_training.drop(['emp_title'], axis = 1)
df_training.drop(['emp_length'], axis = 1)
df_training.drop(['desc'], axis = 1)
df_training.drop(['zip_code'], axis = 1)
df_training.drop(['addr_state'], axis = 1)
df_training.drop(['last_week_pay'], axis = 1)
df_training.drop(['loan_amnt'], axis = 1)
df_training.drop(['funded_amnt'], axis = 1)
#Cleaning Testing data

df_testing.drop(['batch_enrolled'], axis = 1)
df_testing.drop(['emp_title'], axis = 1)
df_testing.drop(['emp_length'], axis = 1)
df_testing.drop(['desc'], axis = 1)
df_testing.drop(['zip_code'], axis = 1)
df_testing.drop(['addr_state'], axis = 1)
df_testing.drop(['last_week_pay'], axis = 1)
df_testing.drop(['loan_amnt'], axis = 1)
df_testing.drop(['funded_amnt'], axis = 1)




#Taking relevant columns- Numeric columns
df_train= df_training[['member_id','grade','funded_amnt_inv','term','int_rate','annual_inc','dti',
        'delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal',
        'revol_util','total_acc','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee',
        'collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt',
        'tot_cur_bal','total_rev_hi_lim']]

df_test= df_testing[['member_id','grade','funded_amnt_inv','term','int_rate','annual_inc','dti',
        'delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal',
        'revol_util','total_acc','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee',
        'collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt',
        'tot_cur_bal','total_rev_hi_lim']] 

#Replacing NaN values with 0
df_train=df_train.fillna(0)
df_test=df_test.fillna(0)
df_test[:10]

,member_id,grade,funded_amnt_inv,term,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,11937648,C,14000.0,60 months,16.24,73500.0,28.65,1.0,1.0,20.0,...,3915.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85230.0,45700.0
1,38983318,B,16000.0,60 months,9.49,105000.0,31.19,0.0,0.0,0.0,...,1495.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,444991.0,21400.0
2,27999917,D,11050.0,60 months,15.61,62300.0,21.73,1.0,0.0,23.0,...,2096.21,0.0,0.0,0.0,0.0,26.0,0.0,0.0,105737.0,16300.0
3,61514932,C,34700.0,60 months,12.69,150000.0,17.27,0.0,0.0,0.0,...,1756.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287022.0,72400.0
4,59622821,A,6500.0,36 months,6.89,52000.0,20.19,0.0,0.0,0.0,...,172.21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,234278.0,26700.0
5,28822038,E,13475.0,60 months,18.99,30000.0,34.80,0.0,0.0,0.0,...,3133.03,0.0,0.0,0.0,0.0,0.0,0.0,131.0,29383.0,42700.0
6,10718089,A,5000.0,36 months,7.62,64000.0,12.30,0.0,0.0,0.0,...,344.69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38403.0,17000.0
7,58114582,F,10000.0,60 months,22.99,36400.0,8.90,0.0,3.0,0.0,...,1110.12,0.0,0.0,0.0,0.0,0.0,0.0,246.0,7119.0,18200.0
8,35023176,B,30000.0,36 months,9.17,110000.0,14.52,1.0,0.0,21.0,...,1386.68,0.0,0.0,0.0,0.0,21.0,0.0,0.0,85611.0,51100.0
9,1268247,C,7000.0,60 months,15.96,130500.0,11.26,0.0,1.0,34.0,...,3051.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_train[:10]

,member_id,grade,funded_amnt_inv,term,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,58189336,E,14350.0,36 months,19.19,28700.0,33.88,0.0,1.0,50.0,...,1173.84,0.0,0.0,0.0,0.0,74.0,0.0,0.0,28699.0,30800.0
1,70011223,B,4800.0,36 months,10.99,65000.0,3.64,0.0,1.0,0.0,...,83.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9974.0,32900.0
2,70255675,A,10000.0,36 months,7.26,45000.0,18.42,0.0,0.0,0.0,...,56.47,0.0,0.0,0.0,0.0,0.0,0.0,65.0,38295.0,34900.0
3,1893936,D,15000.0,36 months,19.72,105000.0,14.97,0.0,2.0,46.0,...,4858.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55564.0,24700.0
4,7652106,B,16000.0,36 months,10.64,52000.0,20.16,0.0,0.0,0.0,...,2296.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47159.0,47033.0
5,10247268,A,14950.0,36 months,8.90,120000.0,12.30,0.0,0.0,56.0,...,1957.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350619.0,29500.0
6,8089625,A,4975.0,36 months,7.90,75000.0,5.70,0.0,0.0,0.0,...,578.36,0.0,0.0,0.0,0.0,0.0,0.0,1023.0,13272.0,55500.0
7,23043116,B,6000.0,36 months,9.17,54000.0,11.63,0.0,1.0,46.0,...,637.51,0.0,0.0,0.0,0.0,54.0,0.0,0.0,272579.0,11800.0
8,45900933,C,6000.0,36 months,13.99,92000.0,30.85,0.0,0.0,77.0,...,621.72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281521.0,62100.0
9,41272507,D,34550.0,60 months,17.14,72000.0,33.92,0.0,0.0,0.0,...,5535.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76034.0,33200.0


In [7]:
df_test.columns

Index(['member_id', 'grade', 'funded_amnt_inv', 'term', 'int_rate',
       'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim'],
      dtype='object')

In [7]:
import re
#Using regex to replace text like months and convert the column , retaining float value.
df_train_=df_train.replace('months',"",regex=True)
df_train_.dtypes
df_train_['term'] = df_train_['term'].astype(float, errors = 'raise')
df_train_.dtypes

df_test_=df_test.replace('months',"",regex=True)

df_test_.dtypes

df_test_['term'] = df_test_['term'].astype(float, errors = 'raise')
df_test_.dtypes


df_train_['member_id']=df_train_['member_id'].astype(str, errors = 'raise')
df_test_['member_id']=df_test_['member_id'].astype(str, errors = 'raise')

#Fill NaN with 0, to handle any empty value after regex process
df_train_=df_train_.fillna(0)
df_test_=df_test_.fillna(0)
df_train_['funded_amnt_inv'] = df_train_['funded_amnt_inv'].div(df_train_['funded_amnt_inv'].sum()).multiply(100)
df_train_['annual_inc'] = df_train_['annual_inc'].div(df_train_['annual_inc'].sum()).multiply(100)
df_train_['total_rec_int'] = df_train_['total_rec_int'].div(df_train_['total_rec_int'].sum()).multiply(100)
df_train_['tot_cur_bal'] = df_train_['tot_cur_bal'].div(df_train_['tot_cur_bal'].sum()).multiply(100)
df_train_['total_rev_hi_lim'] = df_train_['total_rev_hi_lim'].div(df_train_['total_rev_hi_lim'].sum()).multiply(100)
df_train_['tot_coll_amt'] = df_train_['tot_coll_amt'].div(df_train_['tot_coll_amt'].sum()).multiply(100)
df_train_['dti'] = df_train_['dti'].div(df_train_['dti'].sum()).multiply(100)
df_train_['revol_bal'] = df_train_['revol_bal'].div(df_train_['revol_bal'].sum()).multiply(100)
df_train_['revol_util'] = df_train_['revol_util'].div(df_train_['revol_util'].sum()).multiply(100)
df_train_['total_acc'] = df_train_['total_acc'].div(df_train_['total_acc'].sum()).multiply(100)
df_train_['total_rec_late_fee'] = df_train_['total_rec_late_fee'].div(df_train_['total_rec_late_fee'].sum()).multiply(100)
df_train_['recoveries'] = df_train_['recoveries'].div(df_train_['recoveries'].sum()).multiply(100)
df_train_['collection_recovery_fee'] = df_train_['collection_recovery_fee'].div(df_train_['collection_recovery_fee'].sum()).multiply(100)
df_train_['acc_now_delinq'] = df_train_['acc_now_delinq'].div(df_train_['acc_now_delinq'].sum()).multiply(100)
df_train_['tot_cur_bal'] = df_train_['tot_cur_bal'].div(df_train_['tot_cur_bal'].sum()).multiply(100)
df_train_['int_rate'] = df_train_['int_rate'].div(df_train_['int_rate'].sum()).multiply(100)
df_train_['open_acc'] = df_train_['open_acc'].div(df_train_['open_acc'].sum()).multiply(100)
df_train_['pub_rec'] = df_train_['pub_rec'].div(df_train_['pub_rec'].sum()).multiply(100)
df_train_['delinq_2yrs'] = df_train_['delinq_2yrs'].div(df_train_['delinq_2yrs'].sum()).multiply(100)
df_train_['inq_last_6mths'] = df_train_['inq_last_6mths'].div(df_train_['inq_last_6mths'].sum()).multiply(100)
df_train_['acc_now_delinq'] = df_train_['acc_now_delinq'].div(df_train_['acc_now_delinq'].sum()).multiply(100)


df_train_['term'] =df_train_['term'].div(1200).round(2)
df_train_['mths_since_last_delinq'] =df_train_['mths_since_last_delinq'].div(1200).round(5)
df_train_['mths_since_last_major_derog'] =df_train_['mths_since_last_major_derog'].div(1200).round(5)
df_train_['collections_12_mths_ex_med'] =df_train_['collections_12_mths_ex_med'].div(1200).round(5)
df_train_['mths_since_last_delinq'] =df_train_['mths_since_last_delinq'].div(1200).round(5)
df_train_['mths_since_last_record'] =df_train_['mths_since_last_record'].div(1200).round(5)

df_test_['funded_amnt_inv'] = df_test_['funded_amnt_inv'].div(df_test_['funded_amnt_inv'].sum()).multiply(100)
df_test_['annual_inc'] = df_test_['annual_inc'].div(df_test_['annual_inc'].sum()).multiply(100)
df_test_['total_rec_int'] = df_test_['total_rec_int'].div(df_test_['total_rec_int'].sum()).multiply(100)
df_test_['tot_cur_bal'] = df_test_['tot_cur_bal'].div(df_test_['tot_cur_bal'].sum()).multiply(100)
df_test_['total_rev_hi_lim'] = df_test_['total_rev_hi_lim'].div(df_test_['total_rev_hi_lim'].sum()).multiply(100)
df_test_['tot_coll_amt'] = df_test_['tot_coll_amt'].div(df_test_['tot_coll_amt'].sum()).multiply(100)
df_test_['dti'] = df_test_['dti'].div(df_test_['dti'].sum()).multiply(100)
df_test_['revol_bal'] = df_test_['revol_bal'].div(df_test_['revol_bal'].sum()).multiply(100)
df_test_['revol_util'] = df_test_['revol_util'].div(df_test_['revol_util'].sum()).multiply(100)
df_test_['total_acc'] = df_test_['total_acc'].div(df_test_['total_acc'].sum()).multiply(100)
df_test_['total_rec_late_fee'] = df_test_['total_rec_late_fee'].div(df_test_['total_rec_late_fee'].sum()).multiply(100)
df_test_['recoveries'] = df_test_['recoveries'].div(df_test_['recoveries'].sum()).multiply(100)
df_test_['collection_recovery_fee'] = df_test_['collection_recovery_fee'].div(df_test_['collection_recovery_fee'].sum()).multiply(100)
df_test_['acc_now_delinq'] = df_train_['acc_now_delinq'].div(df_test_['acc_now_delinq'].sum()).multiply(100)
df_test_['tot_cur_bal'] = df_test_['tot_cur_bal'].div(df_test_['tot_cur_bal'].sum()).multiply(100)
df_test_['int_rate'] = df_test_['int_rate'].div(df_test_['int_rate'].sum()).multiply(100)
df_test_['open_acc'] = df_test_['open_acc'].div(df_test_['open_acc'].sum()).multiply(100)
df_test_['pub_rec'] = df_test_['pub_rec'].div(df_test_['pub_rec'].sum()).multiply(100)
df_test_['delinq_2yrs'] = df_test_['delinq_2yrs'].div(df_test_['delinq_2yrs'].sum()).multiply(100)
df_test_['inq_last_6mths'] = df_test_['inq_last_6mths'].div(df_test_['inq_last_6mths'].sum()).multiply(100)
df_test_['acc_now_delinq'] = df_test_['acc_now_delinq'].div(df_test_['acc_now_delinq'].sum()).multiply(100)


df_test_['term'] =df_test_['term'].div(1200).round(2)
df_test_['mths_since_last_delinq'] =df_test_['mths_since_last_delinq'].div(1200).round(5)
df_test_['mths_since_last_major_derog'] =df_test_['mths_since_last_major_derog'].div(1200).round(5)
df_test_['collections_12_mths_ex_med'] =df_test_['collections_12_mths_ex_med'].div(1200).round(5)
df_test_['mths_since_last_delinq'] =df_test_['mths_since_last_delinq'].div(1200).round(5)
df_test_['mths_since_last_record'] =df_test_['mths_since_last_record'].div(1200).round(5)
df_train_


,member_id,grade,funded_amnt_inv,term,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,58189336,E,0.000183,0.03,0.000272,0.000072,0.000351,0.0,0.000270,0.00003,...,0.000126,0.0,0.0,0.0,0.0,0.06167,0.0,0.000000,0.000042,0.000196
1,70011223,B,0.000061,0.03,0.000156,0.000163,0.000038,0.0,0.000270,0.00000,...,0.000009,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000015,0.000209
2,70255675,A,0.000128,0.03,0.000103,0.000113,0.000191,0.0,0.000000,0.00000,...,0.000006,0.0,0.0,0.0,0.0,0.00000,0.0,0.000062,0.000056,0.000222
3,1893936,D,0.000192,0.03,0.000280,0.000263,0.000155,0.0,0.000541,0.00003,...,0.000520,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000081,0.000157
4,7652106,B,0.000204,0.03,0.000151,0.000130,0.000209,0.0,0.000000,0.00000,...,0.000246,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000069,0.000299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532423,31296187,B,0.000255,0.03,0.000177,0.000188,0.000150,0.0,0.000000,0.00000,...,0.000278,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000121,0.000158
532424,29403184,C,0.000153,0.05,0.000213,0.000148,0.000238,0.0,0.000000,0.00000,...,0.000234,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000333,0.000113
532425,7357607,E,0.000239,0.05,0.000295,0.000106,0.000282,0.0,0.000270,0.00002,...,0.000069,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000038,0.000154
532426,23182668,D,0.000268,0.05,0.000231,0.000125,0.000154,0.0,0.000270,0.00000,...,0.000495,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000043,0.000148


In [35]:
# With Training set
from sklearn.model_selection import train_test_split

X = df_train_.loc[:, "funded_amnt_inv":,]
y = df_train_.loc[:,  'grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


X_test
y
X

,funded_amnt_inv,term,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,0.000183,0.03,0.000272,0.000072,0.000351,0.0,0.000270,0.00003,0.0625,0.000228,...,0.000126,0.0,0.0,0.0,0.0,0.06167,0.0,0.000000,0.000042,0.000196
1,0.000061,0.03,0.000156,0.000163,0.000038,0.0,0.000270,0.00000,0.0000,0.000098,...,0.000009,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000015,0.000209
2,0.000128,0.03,0.000103,0.000113,0.000191,0.0,0.000000,0.00000,0.0000,0.000081,...,0.000006,0.0,0.0,0.0,0.0,0.00000,0.0,0.000062,0.000056,0.000222
3,0.000192,0.03,0.000280,0.000263,0.000155,0.0,0.000541,0.00003,0.0000,0.000163,...,0.000520,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000081,0.000157
4,0.000204,0.03,0.000151,0.000130,0.000209,0.0,0.000000,0.00000,0.0000,0.000179,...,0.000246,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000069,0.000299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000319,0.05,0.000298,0.000375,0.000148,0.0,0.000270,0.00000,0.0000,0.000163,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000510,0.000126
996,0.000307,0.05,0.000135,0.000155,0.000291,0.0,0.000000,0.00000,0.0000,0.000195,...,0.000177,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000275,0.000331
997,0.000255,0.03,0.000203,0.000133,0.000143,0.0,0.000000,0.00004,0.0000,0.000130,...,0.000497,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000034,0.000243
998,0.000064,0.03,0.000103,0.000133,0.000318,0.0,0.000000,0.00004,0.0000,0.000342,...,0.000016,0.0,0.0,0.0,0.0,0.05333,0.0,0.000000,0.000326,0.000353


In [40]:

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    criterion="gini",
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=1,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight='balanced',
)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)





C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curr

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

In [41]:
y_pred

array(['C', 'D', 'A', 'D', 'D', 'C', 'B', 'D', 'B', 'A', 'D', 'C', 'C',
       'C', 'B', 'A', 'B', 'E', 'E', 'B', 'C', 'D', 'F', 'C', 'C', 'C',
       'B', 'D', 'D', 'C', 'B', 'C', 'B', 'B', 'E', 'E', 'B', 'D', 'B',
       'A', 'B', 'A', 'B', 'C', 'C', 'A', 'D', 'C', 'B', 'D', 'D', 'B',
       'E', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'D', 'E', 'C', 'D', 'C',
       'C', 'A', 'C', 'B', 'C', 'D', 'C', 'D', 'B', 'B', 'C', 'D', 'D',
       'D', 'E', 'D', 'A', 'D', 'A', 'D', 'E', 'D', 'D', 'A', 'C', 'B',
       'B', 'C', 'F', 'A', 'A', 'C', 'D', 'B', 'A', 'D', 'C', 'C', 'C',
       'C', 'A', 'A', 'A', 'B', 'C', 'B', 'B', 'B', 'A', 'A', 'C', 'C',
       'C', 'C', 'B', 'B', 'D', 'F', 'B', 'B', 'C', 'A', 'C', 'F', 'D',
       'C', 'C', 'C', 'C', 'E', 'A', 'D', 'A', 'E', 'E', 'E', 'D', 'F',
       'D', 'D', 'A', 'B', 'D', 'C', 'A', 'B', 'A', 'A', 'B', 'C', 'C',
       'A', 'B', 'B', 'D', 'C', 'D', 'A', 'A', 'F', 'C', 'E', 'A', 'C',
       'B', 'F', 'D', 'A', 'E', 'C', 'B', 'D', 'A', 'F', 'D', 'A

In [42]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           A       0.96      0.96      0.96        53
           B       0.89      0.79      0.84        86
           C       0.78      0.86      0.82        79
           D       0.65      0.86      0.74        42
           E       0.63      0.44      0.52        27
           F       0.80      0.67      0.73        12
           G       0.00      0.00      0.00         1

    accuracy                           0.81       300
   macro avg       0.67      0.65      0.66       300
weighted avg       0.81      0.81      0.81       300



C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [43]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.812793451758969, 0.81, 0.8063663647047773, None)

In [8]:
#with Test Set
from sklearn.model_selection import train_test_split

X = df_test_.loc[:, "funded_amnt_inv":,]
y = df_test_.loc[:,  'grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


X_test
y


0         C
1         B
2         D
3         C
4         A
         ..
354946    A
354947    G
354948    E
354949    A
354950    A
Name: grade, Length: 354951, dtype: object

In [45]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    criterion="gini",
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=1,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight='balanced',
)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)


C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curr

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

...

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

In [47]:
y_pred

array(['B', 'C', 'D', ..., 'A', 'C', 'C'], dtype=object)

In [48]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           A       0.99      0.98      0.98     17678
           B       0.97      0.92      0.94     30492
           C       0.91      0.93      0.92     29606
           D       0.87      0.94      0.90     16780
           E       0.89      0.88      0.88      8532
           F       0.91      0.89      0.90      2699
           G       0.98      0.81      0.88       699

    accuracy                           0.93    106486
   macro avg       0.93      0.91      0.92    106486
weighted avg       0.93      0.93      0.93    106486



In [53]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.9322296784775934, 0.9307796330033995, 0.9310596634495285, None)